In [1]:
import numpy as np
import pandas as pd

In [2]:
file_path = '/Users/mymac/Google Drive/My Drive/Forex_Robot/'

In [3]:
currency_pair = 'Eur_Usd'
rounding = 3 if 'Jpy' in currency_pair else 5

In [4]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_2022-2023.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

df2 = pd.read_csv(file_path + f'Oanda_{currency_pair}_H4_2022-2023.csv')
df2.Date = pd.to_datetime(df2.Date)
df2.reset_index(drop=True, inplace=True)

df3 = pd.read_csv(file_path + f'Oanda_{currency_pair}_D_2022-2023.csv')
df3.Date = pd.to_datetime(df3.Date)
df3.reset_index(drop=True, inplace=True)

In [5]:
def heiken_ashi(open_values, high_values, low_values, close_values):
    ha_close = (open_values + high_values + low_values + close_values) / 4

    ha_open = pd.Series(0.0, index=open_values.index)
    ha_open.iloc[0] = open_values.iloc[0]

    for i in range(1, len(open_values)):
        ha_open.iloc[i] = (ha_open.iloc[i - 1] + ha_close.iloc[i - 1]) / 2

    ha_high = pd.concat([ha_open, ha_close, high_values], axis=1).max(axis=1)
    ha_low = pd.concat([ha_open, ha_close, low_values], axis=1).min(axis=1)

    return ha_open, ha_high, ha_low, ha_close


def trend_alert(df_mid_term, df_long_term, mid_term_ema_len=20):
    ha_open_mid_term, _, _, ha_close_mid_term = heiken_ashi(df_mid_term['Mid_Open'], df_mid_term['Mid_High'], df_mid_term['Mid_Low'], df_mid_term['Mid_Close'])
    ha_open_long_term, _, _, ha_close_long_term = heiken_ashi(df_long_term['Mid_Open'], df_long_term['Mid_High'], df_long_term['Mid_Low'], df_long_term['Mid_Close'])

    long_term_long = ha_close_long_term > ha_open_long_term 
    long_term_short = ha_close_long_term < ha_open_long_term
    long_term_with_date = pd.concat([df_long_term['Date'], long_term_long, long_term_short], axis=1)
    long_term_with_date = long_term_with_date.rename(columns={0: 'long_term_long', 1: 'long_term_short'})

    foo = pd.concat([df_long_term['Date'], ha_open_long_term, ha_close_long_term], axis=1)

    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(foo.iloc[-5:])

    mid_term_ema = pd.Series.ewm(ha_close_mid_term, span=mid_term_ema_len).mean()
    mid_term_ema_delta = mid_term_ema.diff()
    mid_term_long = (ha_close_mid_term > ha_open_mid_term) & (ha_close_mid_term > mid_term_ema) & (mid_term_ema_delta > 0)
    mid_term_short = (ha_close_mid_term < ha_open_mid_term) & (ha_close_mid_term < mid_term_ema) & (mid_term_ema_delta < 0)
    mid_term_with_date = pd.concat([df_mid_term['Date'], mid_term_long, mid_term_short], axis=1)
    mid_term_with_date = mid_term_with_date.rename(columns={0: 'mid_term_long', 1: 'mid_term_short'})

    # foo = pd.concat([df_mid_term['Date'], ha_open_mid_term, ha_close_mid_term, mid_term_ema, mid_term_ema_delta], axis=1)

    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #     print(foo.iloc[-5:])

    terms_with_dates = pd.merge(mid_term_with_date, long_term_with_date, on='Date', how='left')
    terms_with_dates.reset_index(drop=True, inplace=True)
    terms_with_dates = terms_with_dates.fillna(method='ffill')
    terms_with_dates.dropna(inplace=True)
    terms_with_dates.reset_index(drop=True, inplace=True)

    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print()
        print(terms_with_dates[['Date', 'mid_term_long', 'mid_term_short', 'long_term_short']].iloc[-5:])

    is_long = terms_with_dates['mid_term_long'] & terms_with_dates['long_term_long']
    is_short = terms_with_dates['mid_term_short'] & terms_with_dates['long_term_short']

    terms_with_dates['trend_alert'] = np.where(is_long, 1, np.where(is_short, -1, 0))
    terms_with_dates = terms_with_dates[['Date', 'trend_alert']]

    return terms_with_dates

def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).mean()

def atr_bands(high, low, close, lookback=3, atr_multiplier=2.5):
    scaled_atr_vals = atr(high, low, close, lookback) * atr_multiplier
    lower_band = close - scaled_atr_vals
    upper_band = close + scaled_atr_vals

    return lower_band, upper_band

def impulse_macd(high, low, close, ma_len=34, signal_len=9):
    def calc_smma(src, length):
        smma = np.zeros_like(src)
        for i in range(len(src)):
            if np.isnan(smma[i - 1]):
                smma[i] = np.mean(src[i - length + 1 : i + 1])
            else:
                smma[i] = (smma[i - 1] * (length - 1) + src[i]) / length
        return smma

    def calc_zlema(src, length):
        ema1 = pd.Series(src).ewm(span=length).mean()
        ema2 = ema1.ewm(span=length).mean()
        d = ema1 - ema2
        return ema1 + d
    
    src = (high + low + close) / 3 
    hi = calc_smma(high, ma_len)
    lo = calc_smma(low, ma_len)
    mi = calc_zlema(src, ma_len)

    md = np.where(mi > hi, mi - hi, np.where(mi < lo, mi - lo, 0))
    sb = pd.Series(md).rolling(window=signal_len).mean()

    return md, sb

In [6]:
df['lower_atr_band'], df['upper_atr_band'] = atr_bands(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['impulse_macd'], df['impulse_macd_signal'] = impulse_macd(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
trend_alerts = trend_alert(df2, df3)
df = pd.merge(df, trend_alerts, on='Date', how='left')
df.reset_index(drop=True, inplace=True)
df = df.fillna(method='ffill')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)


                   Date         0         1
256 2023-05-14 21:00:00  1.092742  1.086663
257 2023-05-15 21:00:00  1.089702  1.087425
258 2023-05-16 21:00:00  1.088564  1.084665
259 2023-05-17 21:00:00  1.086614  1.080565
260 2023-05-18 21:00:00  1.083590  1.079130

                    Date  mid_term_long  mid_term_short long_term_short
1552 2023-05-18 13:00:00          False            True            True
1553 2023-05-18 17:00:00          False            True            True
1554 2023-05-18 21:00:00          False            True            True
1555 2023-05-19 01:00:00          False            True            True
1556 2023-05-19 05:00:00          False           False            True


In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df[['Date', 'trend_alert']].iloc[-1000:])

                     Date  trend_alert
73518 2023-05-15 18:30:00          0.0
73519 2023-05-15 18:35:00          0.0
73520 2023-05-15 18:40:00          0.0
73521 2023-05-15 18:45:00          0.0
73522 2023-05-15 18:50:00          0.0
73523 2023-05-15 18:55:00          0.0
73524 2023-05-15 19:00:00          0.0
73525 2023-05-15 19:05:00          0.0
73526 2023-05-15 19:10:00          0.0
73527 2023-05-15 19:15:00          0.0
73528 2023-05-15 19:20:00          0.0
73529 2023-05-15 19:25:00          0.0
73530 2023-05-15 19:30:00          0.0
73531 2023-05-15 19:35:00          0.0
73532 2023-05-15 19:40:00          0.0
73533 2023-05-15 19:45:00          0.0
73534 2023-05-15 19:50:00          0.0
73535 2023-05-15 19:55:00          0.0
73536 2023-05-15 20:00:00          0.0
73537 2023-05-15 20:05:00          0.0
73538 2023-05-15 20:10:00          0.0
73539 2023-05-15 20:15:00          0.0
73540 2023-05-15 20:20:00          0.0
73541 2023-05-15 20:25:00          0.0
73542 2023-05-15 20:30:00

In [9]:
# df[df['trend_alert'] == -1]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df[['Date', 'trend_alert']].iloc[72100:])

                     Date  trend_alert
72100 2023-05-08 20:15:00          0.0
72101 2023-05-08 20:20:00          0.0
72102 2023-05-08 20:25:00          0.0
72103 2023-05-08 20:30:00          0.0
72104 2023-05-08 20:35:00          0.0
72105 2023-05-08 20:40:00          0.0
72106 2023-05-08 20:45:00          0.0
72107 2023-05-08 20:50:00          0.0
72108 2023-05-08 20:55:00          0.0
72109 2023-05-08 21:00:00         -1.0
72110 2023-05-08 21:05:00         -1.0
72111 2023-05-08 21:10:00         -1.0
72112 2023-05-08 21:15:00         -1.0
72113 2023-05-08 21:20:00         -1.0
72114 2023-05-08 21:25:00         -1.0
72115 2023-05-08 21:30:00         -1.0
72116 2023-05-08 21:35:00         -1.0
72117 2023-05-08 21:40:00         -1.0
72118 2023-05-08 21:45:00         -1.0
72119 2023-05-08 21:50:00         -1.0
72120 2023-05-08 21:55:00         -1.0
72121 2023-05-08 22:00:00         -1.0
72122 2023-05-08 22:05:00         -1.0
72123 2023-05-08 22:10:00         -1.0
72124 2023-05-08 22:15:00

In [7]:
df['trend_alert'].value_counts()

 1.0    26178
 0.0    26074
-1.0    22266
Name: trend_alert, dtype: int64